In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML

import numpy as np

In [2]:
def make_HTML_table(data):
    table = '<div class="rendered_html jp-RenderedHTMLCommon"><table>\n'

    header = data[0]
    table += "  <tr>\n"
    for column in header:
        table += "    <th>{0}</th>\n".format(column.strip())
    table += "  </tr>\n"

    for row in data[1:]:
        table += "  <tr>\n"
        for column in row:
            table += "    <td>{0}</td>\n".format(column.strip())
        table += "  </tr>\n"

    table += "</table></div>"
    return table

In [3]:
_shift_lookup = {'L': (-1, 0),
                 'R': (1, 0),
                 'U': (0, -1),
                 'D': (0, 1)
                }

class Box():
    def __init__(self, x, y):
        # dimensions
        self.x = x
        self.y = y
        # TL position
        self.i = None
        self.j = None
        
    def all_covered(self, dx=0, dy=0):
        xs = np.arange(self.x, dtype=int) + self.i + dx
        ys = np.arange(self.y, dtype=int) + self.j + dy
        return np.array([(x,y) for x in xs for y in ys])
        
    def set_pos(self, i, j):
        """Top left position of box"""
        self.i = i
        self.j = j
        

In [31]:
# based on https://ipywidgets.readthedocs.io/en/stable/examples/Variable%20Inspector.html
class Updater():
    instance = None

    def __init__(self, ipython, dim, box_data):
        """Public constructor."""
        if Updater.instance is not None:
            raise Exception("""Only one instance can exist at a
                time.  Call close() on the active instance before creating a new instance.
                If you have lost the handle to the active instance, you can re-obtain it
                via `Updater.instance`.""")

        Updater.instance = self
        self.closed = False

        self._box = widgets.Box()
        self._box.layout.overflow = 'visible scroll'
        self._table = widgets.HTML(value = 'Not hooked')
        self._box.children = [self._table]

        self._ipython = ipython
        self._ipython.events.register('post_run_cell', self._fill)

        self.data = np.zeros((dim, dim), dtype=int)
        self.dim = dim
        self.header = [str(i) for i in range(self.data.shape[0])]
        self.boxes = {}
        self.intids = {'-': 0}
        for intid, (label, x, y, i, j) in enumerate(box_data):
            box = Box(x, y)
            box.set_pos(i, j)
            self.boxes[label] = box
            self.intids[label] = intid + 1
            for xc, yc in box.all_covered():
                self.data[yc, xc] = self.intids[label]
        self.labelids = dict([(v, k) for k, v in self.intids.items()])
        self._fill()
        
    def shift(self, label, dirn):
        box = self.boxes[label]
        dx, dy = _shift_lookup[dirn]
        new_i = box.i + dx
        new_j = box.j + dy
        ok = True
        if new_i < 0 or new_i + box.x > self.dim \
          or new_j < 0 or new_j + box.y > self.dim:
            # simple bounds check
            ok = False
        else:
            # check occupancy of target space
            all_hit = box.all_covered(dx, dy)
            if not all([self.data[y, x] in (0, self.intids[label]) for (x,y) in all_hit]):
                ok = False
        if ok:
            for xc, yc in box.all_covered():
                self.data[yc, xc] = self.intids['-']
            box.set_pos(new_i, new_j)
            for xc, yc in all_hit:
                self.data[yc, xc] = self.intids[label]
        else:
            print("Can't move that way")
        
    def set(self, i, j, intid):
        self.data[i,j] = intid
        
    def as_list(self):
        data = [self.header]
        for row in list(self.data):
            data.append([self.labelids[c] for c in row])
        return data
        
    def close(self):
        """Close and remove hooks."""
        if not self.closed:
            # self._ipython.events.unregister('post_run_cell', self._fill)
            # self._table.close()
            self.closed = True
            Updater.instance = None

    def _fill(self):
        """Fill self with variable information."""
        self._table.value = make_HTML_table(self.as_list())
        
    def _repr_html_(self):
        return self._table

    def _ipython_display_(self):
        """Called when display() or pyout is used to display the object.
        """
        self._box._ipython_display_()

In [32]:
box_data = [('A', 1, 1, 0, 0), ('B', 2, 2, 1, 0)]

In [33]:
P = Updater(get_ipython(), 3, box_data)
P

Box(children=(HTML(value='<div class="rendered_html jp-RenderedHTMLCommon"><table>\n  <tr>\n    <th>0</th>\n  …

## Move 'A' or 'B' in a canonical direction, up = 'U', down = 'D', left = 'L', right = 'R'

In [35]:
P.shift('B', 'D')

In [8]:
P.boxes['B'].all_covered()

array([[1, 0],
       [1, 1],
       [2, 0],
       [2, 1]])

In [26]:
P.close()